[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neuromorphs/osn23-huge-tapeout/blob/main/Train_Binarized_SNN.ipynb)

In [ ]:
!pip install snntorch

In [ ]:
import torch
config = {
        'model' : 'Net',

        'exp_name' : 'mnist_tha',
        'num_trials' : 5,
        'num_epochs' : 500,
        'binarize' : True,
        'data_dir' : "~/data/mnist",
        'batch_size' : 128,
        'seed' : 0,
        'num_workers' : 0,

        # final run sweeps
        'save_csv' : True,
        'save_model' : True,
        'early_stopping': True,
        'patience': 100,

        # final params
        'grad_clip' : False,
        'weight_clip' : False,
        'batch_norm' : True,
        'dropout1' : 0.02856,
        'beta' : 0.99,
        'lr' : 9.97e-3,
        'slope': 10.22,

        # threshold annealing. note: thr_final = threshold + thr_final
        'threshold1' : 11.666,
        'alpha_thr1' : 0.024,
        'thr_final1' : 4.317,

        'threshold2' : 14.105,
        'alpha_thr2' : 0.119,
        'thr_final2' : 16.29,

        'threshold3' : 0.6656,
        'alpha_thr3' : 0.0011,
        'thr_final3' : 3.496,

        # fixed params
        'num_steps' : 100,
        'correct_rate': 0.8,
        'incorrect_rate' : 0.2,
        'betas' : (0.9, 0.999),
        't_0' : 4688,
        'eta_min' : 0,
        'df_lr' : True, # return learning rate. Useful for scheduling
    }

def optim_func(net, config):
    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"], betas=config['betas'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['t_0'], eta_min=config['eta_min'], last_epoch=-1)
    return optimizer, scheduler


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Function

class BinarizeF(Function):
    @staticmethod
    def forward(ctx, input):
        output = input.new(input.size())
        output[input >= 0] = 1
        output[input < 0] = -1
        return output

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()
        return grad_input

# aliases
binarize = BinarizeF.apply

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class BinaryTanh(nn.Module):
    def __init__(self):
        super(BinaryTanh, self).__init__()
        self.hardtanh = nn.Hardtanh()

    def forward(self, input):
        output = self.hardtanh(input)
        output = binarize(output)
        return output


class BinaryLinear(nn.Linear):

    def forward(self, input):
        binary_weight = binarize(self.weight)
        if self.bias is None:
            return F.linear(input, binary_weight)
        else:
            return F.linear(input, binary_weight, self.bias)

    def reset_parameters(self):
        # Glorot initialization
        in_features, out_features = self.weight.size()
        stdv = math.sqrt(1.5 / (in_features + out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

        self.weight.lr_scale = 1. / stdv

class SparseBinaryLinear(nn.Linear):
    def __init__(self, in_features, out_features, sparsity=0, bias=True, device=None, dtype=None):
        super(SparseBinaryLinear, self).__init__(in_features, out_features, bias, device, dtype)
        self.mask = torch.bernoulli(torch.ones_like(self.weight) * (1-sparsity))
        self.register_buffer('weight_mask_const', self.mask)

    def forward(self, input):
        binary_weight = binarize(self.weight).mul(Variable(self.weight_mask_const))
        if self.bias is None:
            return F.linear(input, binary_weight)
        else:
            return F.linear(input, binary_weight, self.bias)

    def reset_parameters(self):
        # Glorot initialization
        in_features, out_features = self.weight.size()
        stdv = math.sqrt(1.5 / (in_features + out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

        self.weight.lr_scale = 1. / stdv

class BinaryConv2d(nn.Conv2d):

    def forward(self, input):
        bw = binarize(self.weight)
        return F.conv2d(input, bw, self.bias, self.stride,
                               self.padding, self.dilation, self.groups)

    def reset_parameters(self):
        # Glorot initialization
        in_features = self.in_channels
        out_features = self.out_channels
        for k in self.kernel_size:
            in_features *= k
            out_features *= k
        stdv = math.sqrt(1.5 / (in_features + out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

        self.weight.lr_scale = 1. / stdv

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

def load_data(config):
        data_dir = config['data_dir']

        transform = transforms.Compose([
                transforms.Resize((28, 28)),
                transforms.Grayscale(),
                transforms.ToTensor(),
                transforms.Normalize((0,), (1,))])

        trainset = datasets.MNIST(data_dir, train=True, download=True, transform=transform)
        testset = datasets.MNIST(data_dir, train=False, download=True, transform=transform)

        return trainset, testset

In [ ]:
import torch


class EarlyStopping_acc:
    """Early stops the training if test acc doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.test_loss_min = 0
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, test_loss, model):

        score = test_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(test_loss, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                self.counter = 0
        else:
            self.best_score = score
            self.save_checkpoint(test_loss, model)
            self.counter = 0

    def save_checkpoint(self, test_loss, model):
        '''Saves model when test acc increases.'''
        if self.verbose:
            self.trace_func(f'Test acc increased ({self.test_loss_min:.6f} --> {test_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.test_loss_min = test_loss

In [ ]:
import torch
import snntorch as snn
from snntorch import functional as SF

def test_accuracy(config, net, testloader, device="cpu"):
    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs, _ = net(images)
            accuracy = SF.accuracy_rate(outputs, labels)

            total += labels.size(0)
            correct += accuracy * labels.size(0)

    return 100 * correct / total

In [ ]:
# exp relaxation implementation of THA based on Eq (4)

def thr_annealing(config, network):
    alpha_thr1 = config['alpha_thr1']
    alpha_thr2 = config['alpha_thr2']
    alpha_thr3 = config['alpha_thr3']

    thr_final1 = config['thr_final1']
    thr_final2 = config['thr_final2']
    thr_final3 = config['thr_final3']

    network.lif1.threshold += (thr_final1 - network.lif1.threshold) * alpha_thr1
    network.lif2.threshold += (thr_final2 - network.lif2.threshold) * alpha_thr2
    network.lif3.threshold += (thr_final3 - network.lif3.threshold) * alpha_thr3

    return

In [ ]:
# snntorch
import snntorch as snn
from snntorch import spikegen
from snntorch import surrogate
from snntorch import functional as SF

# torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# misc
import os
import numpy as np
import math
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import time
from tqdm import tqdm

def train(config, net, epoch, trainloader, testloader, criterion, optimizer, scheduler, device):
    net.train()
    loss_accum = []
    lr_accum = []

    # TRAIN
    progress_bar = tqdm(trainloader)
    loss_current = None

    #for data, labels in trainloader:
    for data, labels in progress_bar:
        data, labels = data.to(device), labels.to(device)

        spk_rec2, _ = net(data)
        loss = criterion(spk_rec2, labels)
        optimizer.zero_grad()
        loss.backward()
        if loss_current is None:
            loss_current = loss.item()
        else:
            loss_current = 0.9 * loss_current + 0.1 * loss.item()
        progress_bar.set_description(f"loss: {loss_current:.4f}")

        if config['grad_clip']:
            nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        if config['weight_clip']:
            with torch.no_grad():
                for param in net.parameters():
                    param.clamp_(-1, 1)

        optimizer.step()
        scheduler.step()
        thr_annealing(config, net)


        loss_accum.append(loss.item()/config['num_steps'])
        lr_accum.append(optimizer.param_groups[0]["lr"])


    return loss_accum, lr_accum

In [ ]:
# snntorch
import snntorch as snn
from snntorch import spikegen
from snntorch import surrogate

# torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# misc
import numpy as np
import pandas as pd
import time
import logging

def run(config):
    print(config)
    file_name = config['exp_name']

    ### to address conditional parameters, s.t. thr_final > threshold
    config['thr_final1'] = config['thr_final1'] + config['threshold1']
    config['thr_final2'] = config['thr_final2'] + config['threshold2']
    config['thr_final3'] = config['thr_final3'] + config['threshold3']

    threshold1 = config['threshold1']
    threshold2 = config['threshold2']
    threshold3 = config['threshold3']

    for trial in range(config['num_trials']):
        # file names
        SAVE_CSV = config['save_csv']
        SAVE_MODEL = config['save_model']
        csv_name = file_name + '_t' + str(trial) + '.csv'
        log_name = file_name + '_t' + str(trial) + '.log'
        model_name = file_name + '_t' + str(trial) + '.pt'
        num_epochs = config['num_epochs']
        torch.manual_seed(config['seed'])

        config['threshold1'] = threshold1
        config['threshold2'] = threshold2
        config['threshold3'] = threshold3

        # dataframes
        df_train_loss = pd.DataFrame()
        df_test_acc = pd.DataFrame(columns=['epoch', 'test_acc', 'train_time'])
        df_lr = pd.DataFrame()

        # initialize network
        net = None
        net_desc = config['model']
        if net_desc in globals():
            klass = globals()[net_desc]
            net = klass(config)
        else:
            net = eval(net_desc)
        if trial == 0:
            print(net)
        device = "cpu"
        #device = "mps"
        if torch.cuda.is_available():
            device = "cuda:0"
            if torch.cuda.device_count() > 1:
                net = nn.DataParallel(net)
        net.to(device)

        # net params
        criterion = SF.mse_count_loss(correct_rate=config['correct_rate'], incorrect_rate=config['incorrect_rate'])
        optimizer, scheduler = optim_func(net, config)

        # early stopping condition
        if config['early_stopping']:
            early_stopping = EarlyStopping_acc(patience=config['patience'], verbose=True, path=model_name)
            early_stopping.early_stop = False
            early_stopping.best_score = None

        # load data
        trainset, testset = load_data(config)
        config['dataset_length'] = len(trainset)
        trainloader = DataLoader(trainset, batch_size=int(config["batch_size"]), shuffle=True)
        testloader = DataLoader(testset, batch_size=int(config["batch_size"]), shuffle=False)

        print(f"=======Trial: {trial}=======")

        for epoch in range(num_epochs):

            # train
            start_time = time.time()
            loss_list, lr_list = train(config, net, epoch, trainloader, testloader, criterion, optimizer, scheduler, device)
            epoch_time = time.time() - start_time

            # test
            test_acc = test_accuracy(config, net, testloader, device)
            print(f'Epoch: {epoch} \tTest Accuracy: {test_acc}')

            if config['df_lr']:
                df_lr = pd.concat([df_lr, pd.DataFrame(lr_list)])
            df_train_loss = pd.concat([df_train_loss, pd.DataFrame(loss_list)])
            test_data = pd.DataFrame([[epoch, test_acc, epoch_time]], columns = ['epoch', 'test_acc', 'train_time'])
            df_test_acc = pd.concat([df_test_acc, test_data])

            if SAVE_CSV:
                df_train_loss.to_csv('loss_' + csv_name, index=False)
                df_test_acc.to_csv('acc_' + csv_name, index=False)
                if config['df_lr']:
                    df_lr.to_csv('lr_' + csv_name, index=False)

            if config['early_stopping']:
                early_stopping(test_acc, net)

                if early_stopping.early_stop:
                    print("Early stopping")
                    early_stopping.early_stop = False
                    early_stopping.best_score = None
                    break

            if SAVE_MODEL and not config['early_stopping']:
                torch.save(net.state_dict(), model_name)

            #for param in net.parameters():
            #    print(param)
        # net.load_state_dict(torch.load(model_name))


In [ ]:
# snntorch
import snntorch as snn
from snntorch import surrogate

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.thr1 = config['threshold1']
        self.thr2 = config['threshold2']
        self.thr3 = config['threshold3']
        slope = config['slope']
        beta = config['beta']
        self.num_steps = config['num_steps']
        self.batch_norm = config['batch_norm']
        p1 = config['dropout1']
        self.binarize = config['binarize']

        spike_grad = surrogate.fast_sigmoid(slope)
        # Initialize layers with spike operator
        self.bconv1 = BinaryConv2d(1, 16, 5, bias=False)
        self.conv1 = nn.Conv2d(1, 16, 5, bias=False)
        self.conv1_bn = nn.BatchNorm2d(16)
        self.lif1 = snn.Leaky(beta, threshold=self.thr1, spike_grad=spike_grad)
        self.bconv2 = BinaryConv2d(16, 64, 5, bias=False)
        self.conv2 = nn.Conv2d(16, 64, 5, bias=False)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.lif2 = snn.Leaky(beta, threshold=self.thr2, spike_grad=spike_grad)
        self.bfc1 = BinaryLinear(64 * 4 * 4, 10)
        self.fc1 = nn.Linear(64 * 4 * 4, 10)
        self.lif3 = snn.Leaky(beta, threshold=self.thr3, spike_grad=spike_grad)
        self.dropout = nn.Dropout(p1)

    def forward(self, x):

        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        # Binarized
        if self.binarize:

            for step in range(self.num_steps):

                cur1 = F.avg_pool2d(self.bconv1(x), 2)
                if self.batch_norm:
                    cur1 = self.conv1_bn(cur1)
                spk1, mem1 = self.lif1(cur1, mem1)
                cur2 = F.avg_pool2d(self.bconv2(spk1), 2)
                if self.batch_norm:
                    cur2 = self.conv2_bn(cur2)
                spk2, mem2 = self.lif2(cur2, mem2)
                cur3 = self.dropout(self.bfc1(spk2.flatten(1)))
                spk3, mem3 = self.lif3(cur3, mem3)

                spk3_rec.append(spk3)
                mem3_rec.append(mem3)

            return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

        # Full Precision
        else:

            for step in range(self.num_steps):

                cur1 = F.avg_pool2d(self.conv1(x), 2)
                if self.batch_norm:
                    cur1 = self.conv1_bn(cur1)
                spk1, mem1 = self.lif1(cur1, mem1)
                cur2 = F.avg_pool2d(self.conv2(spk1), 2)
                if self.batch_norm:
                    cur2 = self.conv2_bn(cur2)
                spk2, mem2 = self.lif2(cur2, mem2)
                cur3 = self.dropout(self.fc1(spk2.flatten(1)))
                spk3, mem3 = self.lif3(cur3, mem3)

                spk3_rec.append(spk3)
                mem3_rec.append(mem3)

            return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

class NetFC(nn.Module):
    def __init__(self, config, neurons=[256, 128], sparsity=0.0):
        super().__init__()

        self.thr1 = config['threshold1']
        self.thr2 = config['threshold2']
        self.thr3 = config['threshold3']
        slope = config['slope']
        beta = config['beta']
        self.num_steps = config['num_steps']
        self.batch_norm = config['batch_norm']
        p1 = config['dropout1']
        self.binarize = config['binarize']

        spike_grad = surrogate.fast_sigmoid(slope)
        # Initialize layers with spike operator
        self.bfc1 = SparseBinaryLinear(28 * 28, neurons[0], sparsity, bias=False)
        self.fc1 = nn.Linear(28 * 28, neurons[0], bias=False)
        self.bn1 = nn.BatchNorm1d(neurons[0])
        self.lif1 = snn.Leaky(beta, threshold=self.thr1, spike_grad=spike_grad)

        self.bfc2 = SparseBinaryLinear(neurons[0], neurons[1], sparsity, bias=False)
        self.fc2 = nn.Linear(neurons[0], neurons[1], bias=False)
        self.bn2 = nn.BatchNorm1d(neurons[1])
        self.lif2 = snn.Leaky(beta, threshold=self.thr2, spike_grad=spike_grad)

        self.bfc3 = BinaryLinear(neurons[1], 10, bias=False)
        self.fc3 = nn.Linear(neurons[1], 10, bias=False)
        self.lif3 = snn.Leaky(beta, threshold=self.thr3, spike_grad=spike_grad)
        self.dropout = nn.Dropout(p1)

    def forward(self, x):

        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        # Binarized
        if self.binarize:
            for step in range(self.num_steps):

                cur1 = self.bfc1(x.flatten(1))
                if self.batch_norm:
                    cur1 = self.bn1(cur1)
                spk1, mem1 = self.lif1(cur1, mem1)
                cur2 = self.bfc2(spk1)
                if self.batch_norm:
                    cur2 = self.bn2(cur2)
                spk2, mem2 = self.lif2(cur2, mem2)
                cur3 = self.dropout(self.bfc3(spk2))
                spk3, mem3 = self.lif3(cur3, mem3)
                # =======Trial: 0=======
                # loss: 0.8453: 100%|██████████| 469/469 [02:46<00:00,  2.82it/s]
                # Epoch: 0 	Test Accuracy: 91.18
                # Test acc increased (0.000000 --> 91.180000).  Saving model ...
                # loss: 0.6933: 100%|██████████| 469/469 [02:45<00:00,  2.83it/s]
                # Epoch: 1 	Test Accuracy: 92.05
                # Test acc increased (91.180000 --> 92.050000).  Saving model ...
                # loss: 0.6445: 100%|██████████| 469/469 [02:45<00:00,  2.83it/s]
                # Epoch: 2 	Test Accuracy: 93.22
                # Test acc increased (92.050000 --> 93.220000).  Saving model ...
                # loss: 0.5774: 100%|██████████| 469/469 [02:46<00:00,  2.82it/s]
                # Epoch: 3 	Test Accuracy: 94.03
                # Test acc increased (93.220000 --> 94.030000).  Saving model ...
                # loss: 0.5002: 100%|██████████| 469/469 [02:45<00:00,  2.83it/s]
                # Epoch: 4 	Test Accuracy: 94.21
                # Test acc increased (94.030000 --> 94.210000).  Saving model ...
                # loss: 0.5120: 100%|██████████| 469/469 [02:45<00:00,  2.83it/s]
                # Epoch: 5 	Test Accuracy: 94.88
                # Test acc increased (94.210000 --> 94.880000).  Saving model ...
                # loss: 0.4805: 100%|██████████| 469/469 [02:44<00:00,  2.84it/s]
                # Epoch: 6 	Test Accuracy: 95.07
                # Test acc increased (94.880000 --> 95.070000).  Saving model ...
                # loss: 0.4456: 100%|██████████| 469/469 [02:45<00:00,  2.84it/s]
                # Epoch: 7 	Test Accuracy: 95.49
                # Test acc increased (95.070000 --> 95.490000).  Saving model ...
                # loss: 0.4310: 100%|██████████| 469/469 [02:44<00:00,  2.85it/s]
                # Epoch: 8 	Test Accuracy: 95.43
                # EarlyStopping counter: 1 out of 100
                # loss: 0.4317: 100%|██████████| 469/469 [02:42<00:00,  2.89it/s]
                # Epoch: 9 	Test Accuracy: 95.63

                # spk1, mem1 = self.lif1(self.bfc1(x.flatten(1)), mem1)
                # if self.batch_norm:
                #     spk1 = self.bn1(spk1)
                # spk2, mem2 = self.lif2(self.bfc2(spk1), mem2)
                # if self.batch_norm:
                #     spk2 = self.bn2(spk2)
                # cur3 = self.dropout(self.bfc3(spk2))
                # spk3, mem3 = self.lif3(cur3, mem3)
                #                 =======Trial: 0=======
                # loss: 6.6307: 100%|██████████| 469/469 [02:45<00:00,  2.84it/s]
                # Epoch: 0 	Test Accuracy: 62.82
                # Test acc increased (0.000000 --> 62.820000).  Saving model ...
                # loss: 6.2358: 100%|██████████| 469/469 [02:45<00:00,  2.84it/s]
                # Epoch: 1 	Test Accuracy: 65.86
                # Test acc increased (62.820000 --> 65.860000).  Saving model ...
                # loss: 2.3662: 100%|██████████| 469/469 [02:45<00:00,  2.84it/s]
                # Epoch: 2 	Test Accuracy: 84.27
                # Test acc increased (65.860000 --> 84.270000).  Saving model ...
                # loss: 1.1735: 100%|██████████| 469/469 [02:46<00:00,  2.82it/s]
                # Epoch: 3 	Test Accuracy: 89.85
                # Test acc increased (84.270000 --> 89.850000).  Saving model ...
                # loss: 0.9328: 100%|██████████| 469/469 [02:45<00:00,  2.83it/s]
                # Epoch: 4 	Test Accuracy: 90.92
                # Test acc increased (89.850000 --> 90.920000).  Saving model ...
                # loss: 0.8480: 100%|██████████| 469/469 [02:46<00:00,  2.82it/s]

                spk3_rec.append(spk3)
                mem3_rec.append(mem3)

            return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

        # Full Precision
        else:

            for step in range(self.num_steps):

                cur1 = self.fc1(x.flatten(1))
                if self.batch_norm:
                    cur1 = self.bn1(cur1)
                spk1, mem1 = self.lif1(cur1, mem1)
                cur2 = self.fc2(spk1)
                if self.batch_norm:
                    cur2 = self.bn2(cur2)
                spk2, mem2 = self.lif2(cur2, mem2)
                cur3 = self.dropout(self.fc3(spk2))
                spk3, mem3 = self.lif3(cur3, mem3)

                spk3_rec.append(spk3)
                mem3_rec.append(mem3)

            return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

class NetFC_FirstConv(nn.Module):
    def __init__(self, config, neurons=[16, 256]):
        super().__init__()

        self.thr1 = config['threshold1']
        self.thr2 = config['threshold2']
        self.thr3 = config['threshold3']
        slope = config['slope']
        beta = config['beta']
        self.num_steps = config['num_steps']
        self.batch_norm = config['batch_norm']
        p1 = config['dropout1']
        self.binarize = config['binarize']

        spike_grad = surrogate.fast_sigmoid(slope)
        # Initialize layers with spike operator
        self.bconv1 = BinaryConv2d(1, neurons[0], 5, bias=False)
        self.conv1 = nn.Conv2d(1, neurons[0], 5, bias=False)
        self.conv1_bn = nn.BatchNorm2d(neurons[0])
        self.lif1 = snn.Leaky(beta, threshold=self.thr1, spike_grad=spike_grad)

        n = ((28-(5//2))/2)**2
        print(n)
        self.bfc2 = BinaryLinear(n, neurons[1])
        self.fc2 = nn.Linear(n, neurons[1])
        self.bn2 = nn.BatchNorm1d(neurons[1])
        self.lif2 = snn.Leaky(beta, threshold=self.thr2, spike_grad=spike_grad)

        self.bfc3 = BinaryLinear(neurons[1], 10)
        self.fc3 = nn.Linear(neurons[1], 10)
        self.lif3 = snn.Leaky(beta, threshold=self.thr3, spike_grad=spike_grad)
        self.dropout = nn.Dropout(p1)

    def forward(self, x):

        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        # Binarized
        if self.binarize:

            for step in range(self.num_steps):

                cur1 = F.avg_pool2d(self.bconv1(x), 2)
                if self.batch_norm:
                    cur1 = self.conv1_bn(cur1)
                spk1, mem1 = self.lif1(cur1, mem1)
                spk2, mem2 = self.lif2(self.bfc2(spk1.flatten(1)), mem2)
                if self.batch_norm:
                    spk2 = self.bn2(spk2)
                cur3 = self.dropout(self.bfc3(spk2))
                spk3, mem3 = self.lif3(cur3, mem3)

                spk3_rec.append(spk3)
                mem3_rec.append(mem3)

            return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

        # Full Precision
        else:

            for step in range(self.num_steps):

                cur1 = F.avg_pool2d(self.conv1(x), 2)
                if self.batch_norm:
                    cur1 = self.conv1_bn(cur1)
                spk1, mem1 = self.lif1(cur1, mem1)
                spk2, mem2 = self.lif2(self.fc2(spk1.flatten(1)), mem2)
                if self.batch_norm:
                    spk2 = self.bn2(spk2)
                cur3 = self.dropout(self.fc3(spk2))
                spk3, mem3 = self.lif3(cur3, mem3)

                spk3_rec.append(spk3)
                mem3_rec.append(mem3)

            return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

In [ ]:
cfg = config
cfg['model'] = 'NetFC'
run(cfg)

{'model': 'NetFC', 'exp_name': 'mnist_tha', 'num_trials': 5, 'num_epochs': 500, 'binarize': True, 'data_dir': '~/data/mnist', 'batch_size': 128, 'seed': 0, 'num_workers': 0, 'save_csv': True, 'save_model': True, 'early_stopping': True, 'patience': 100, 'grad_clip': False, 'weight_clip': False, 'batch_norm': True, 'dropout1': 0.02856, 'beta': 0.99, 'lr': 0.00997, 'slope': 10.22, 'threshold1': 11.666, 'alpha_thr1': 0.024, 'thr_final1': 4.317, 'threshold2': 14.105, 'alpha_thr2': 0.119, 'thr_final2': 16.29, 'threshold3': 0.6656, 'alpha_thr3': 0.0011, 'thr_final3': 3.496, 'num_steps': 100, 'correct_rate': 0.8, 'incorrect_rate': 0.2, 'betas': (0.9, 0.999), 't_0': 4688, 'eta_min': 0, 'df_lr': True}
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
     

loss: 0.7371: 100%|██████████| 469/469 [02:37<00:00,  2.97it/s]


Epoch: 0 	Test Accuracy: 95.05
Test acc increased (0.000000 --> 95.050000).  Saving model ...


loss: 0.6158: 100%|██████████| 469/469 [02:35<00:00,  3.02it/s]


Epoch: 1 	Test Accuracy: 95.83
Test acc increased (95.050000 --> 95.830000).  Saving model ...


...

loss: 0.3943: 100%|██████████| 469/469 [02:35<00:00,  3.01it/s]


Epoch: 12 	Test Accuracy: 96.76
EarlyStopping counter: 3 out of 100


loss: 0.3945: 100%|██████████| 469/469 [02:35<00:00,  3.01it/s]


Epoch: 13 	Test Accuracy: 97.16
EarlyStopping counter: 4 out of 100


loss: 0.4393:  91%|█████████ | 427/469 [02:21<00:14,  2.93it/s]

In [ ]:
cfg = config
cfg['model'] = 'NetFC(config, sparsity=0.1)'
run(cfg)

In [ ]:
cfg = config
cfg['model'] = 'NetFC(config, [128, 128], sparsity=0.1)'
run(cfg)

{'model': 'NetFC(config, [128, 128], sparsity=0.1)', 'exp_name': 'mnist_tha', 'num_trials': 5, 'num_epochs': 500, 'binarize': True, 'data_dir': '~/data/mnist', 'batch_size': 128, 'seed': 0, 'num_workers': 0, 'save_csv': True, 'save_model': True, 'early_stopping': True, 'patience': 100, 'grad_clip': False, 'weight_clip': False, 'batch_norm': True, 'dropout1': 0.02856, 'beta': 0.99, 'lr': 0.00997, 'slope': 10.22, 'threshold1': 11.666, 'alpha_thr1': 0.024, 'thr_final1': 39.315, 'threshold2': 14.105, 'alpha_thr2': 0.119, 'thr_final2': 58.605000000000004, 'threshold3': 0.6656, 'alpha_thr3': 0.0011, 'thr_final3': 5.492799999999999, 'num_steps': 100, 'correct_rate': 0.8, 'incorrect_rate': 0.2, 'betas': (0.9, 0.999), 't_0': 4688, 'eta_min': 0, 'df_lr': True, 'dataset_length': 60000}
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.],
 

loss: 2.2480: 100%|██████████| 469/469 [02:39<00:00,  2.94it/s]


Epoch: 0 	Test Accuracy: 68.11
Test acc increased (0.000000 --> 68.110000).  Saving model ...


loss: 1.6682: 100%|██████████| 469/469 [02:39<00:00,  2.93it/s]


Epoch: 1 	Test Accuracy: 82.4
Test acc increased (68.110000 --> 82.400000).  Saving model ...


loss: 1.3879: 100%|██████████| 469/469 [02:41<00:00,  2.90it/s]


Epoch: 2 	Test Accuracy: 86.15
Test acc increased (82.400000 --> 86.150000).  Saving model ...


loss: 1.3864:  32%|███▏      | 148/469 [00:51<01:51,  2.87it/s]


KeyboardInterrupt: ignored

In [ ]:
cfg = config
cfg['model'] = 'NetFC(config, [128, 128])'
run(cfg)

{'model': 'NetFC(config, [128, 128])', 'exp_name': 'mnist_tha', 'num_trials': 5, 'num_epochs': 500, 'binarize': True, 'data_dir': '~/data/mnist', 'batch_size': 128, 'seed': 0, 'num_workers': 0, 'save_csv': True, 'save_model': True, 'early_stopping': True, 'patience': 100, 'grad_clip': False, 'weight_clip': False, 'batch_norm': True, 'dropout1': 0.02856, 'beta': 0.99, 'lr': 0.00997, 'slope': 10.22, 'threshold1': 11.666, 'alpha_thr1': 0.024, 'thr_final1': 4.317, 'threshold2': 14.105, 'alpha_thr2': 0.119, 'thr_final2': 16.29, 'threshold3': 0.6656, 'alpha_thr3': 0.0011, 'thr_final3': 3.496, 'num_steps': 100, 'correct_rate': 0.8, 'incorrect_rate': 0.2, 'betas': (0.9, 0.999), 't_0': 4688, 'eta_min': 0, 'df_lr': True}
NetFC(
  (bfc1): SparseBinaryLinear(in_features=784, out_features=128, bias=True)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lif1): Leaky()
  (bfc2): SparseBinaryLinea

100%|██████████| 9912422/9912422 [00:00<00:00, 81587872.50it/s]


Extracting /root/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 102831658.59it/s]

Extracting /root/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/data/mnist/MNIST/raw



...

Epoch: 9 	Test Accuracy: 96.15
Test acc increased (95.810000 --> 96.150000).  Saving model ...


loss: 0.5021: 100%|██████████| 469/469 [02:34<00:00,  3.04it/s]


Epoch: 10 	Test Accuracy: 96.11
EarlyStopping counter: 1 out of 100


loss: 0.5650:   7%|▋         | 34/469 [00:11<02:25,  2.99it/s]


KeyboardInterrupt: ignored

In [ ]:
binarize = BinarizeAnsSparsifyF.apply; print(binarize)
cfg = config
cfg['model'] = 'NetFC(config, [128, 128])'
run(cfg)
binarize = BinarizeF.apply

<bound method Function.apply of <class '__main__.BinarizeAnsSparsifyF'>>
{'model': 'NetFC(config, [128, 128])', 'exp_name': 'mnist_tha', 'num_trials': 5, 'num_epochs': 500, 'binarize': True, 'data_dir': '~/data/mnist', 'batch_size': 128, 'seed': 0, 'num_workers': 0, 'save_csv': True, 'save_model': True, 'early_stopping': True, 'patience': 100, 'grad_clip': False, 'weight_clip': False, 'batch_norm': True, 'dropout1': 0.02856, 'beta': 0.99, 'lr': 0.00997, 'slope': 10.22, 'threshold1': 11.666, 'alpha_thr1': 0.024, 'thr_final1': 132.64299999999997, 'threshold2': 14.105, 'alpha_thr2': 0.119, 'thr_final2': 171.445, 'threshold3': 0.6656, 'alpha_thr3': 0.0011, 'thr_final3': 10.817599999999995, 'num_steps': 100, 'correct_rate': 0.8, 'incorrect_rate': 0.2, 'betas': (0.9, 0.999), 't_0': 4688, 'eta_min': 0, 'df_lr': True, 'dataset_length': 60000}
NetFC(
  (bfc1): BinaryLinear(in_features=784, out_features=128, bias=True)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (bn1): BatchN

loss: 3.1639: 100%|██████████| 469/469 [02:50<00:00,  2.76it/s]


Epoch: 0 	Test Accuracy: 19.18
Test acc increased (0.000000 --> 19.180000).  Saving model ...


loss: 2.9104: 100%|██████████| 469/469 [02:48<00:00,  2.78it/s]


Epoch: 1 	Test Accuracy: 50.52
Test acc increased (19.180000 --> 50.520000).  Saving model ...


...

loss: 0.4017: 100%|██████████| 469/469 [02:44<00:00,  2.84it/s]


Epoch: 164 	Test Accuracy: 95.55
EarlyStopping counter: 15 out of 100


loss: 0.3431: 100%|██████████| 469/469 [02:41<00:00,  2.90it/s]


Epoch: 165 	Test Accuracy: 95.7
EarlyStopping counter: 16 out of 100


loss: 0.3536:  54%|█████▎    | 252/469 [01:26<01:11,  3.03it/s]

In [ ]:
binarize = BinarizeF.apply; print(binarize)
cfg = config
cfg['model'] = 'NetFC(config, [128, 128])'
run(cfg)

<bound method Function.apply of <class '__main__.BinarizeF'>>


In [ ]:
cfg = config
cfg['model_class_name'] = 'Net'
run(cfg)

In [ ]:
cfg = config
cfg['model_class_name'] = 'NetFC_FirstConv'
run(cfg)

In [ ]:
cfg = config
cfg['model_class_name'] = 'NetFC_FirstConv'
run(cfg)


In [ ]:
cfg = config
cfg['model'] = 'NetFC_FirstConv(config)'
run(config=cfg)